## Dependencies

In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from xgboost import XGBRegressor

In [ ]:
raw = pd.DataFrame(pd.read_csv(
    "/kaggle/input/can-humans-really-be-random/survey.csv"
).drop(
    "Timestamp", axis=1
).values.reshape(-1, 1), columns=["0"])

for i in range(1, 21):
    raw.insert(i, column=str(i), value=raw.iloc(axis=1)[i-1])
for column in raw:
    raw[column] = raw[column].shift(list(raw.columns).index(column))

raw = raw.dropna()
raw = raw.reset_index().drop("index", axis=1).astype(np.int)

raw

In [ ]:
scaler = MinMaxScaler()
raw = pd.DataFrame(scaler.fit_transform(raw.values.reshape(-1, 1)).reshape(raw.values.shape)).drop(20, axis=1)
raw

In [ ]:
X = raw.iloc(axis=1)[0: 19]
y = raw.iloc(axis=1)[19]

X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.style import use

use("dark_background")

# Question Answer Distribution
Below is the visualization of the question answers.

In [ ]:
fig, ax = plt.subplots(5, 4)
fig.set_size_inches((24, 18))
fig.subplots_adjust(hspace=0.6, wspace=0.6)
fig.patch.set_facecolor('#333')


plots = 0
cols = ["aquamarine", "azure", "gold", "coral", "wheat", "purple"] * 4


bins = 10
for i in range(5):
    for j in range(4):
        ax[i, j].hist(raw[raw.columns[plots]], bins=bins, color=str("xkcd:" + cols[plots]))
        
        ax[i, j].title.set_text(f"Question: {raw.columns[plots]}")
        ax[i, j].grid()
        ax[i, j].set_axisbelow(False)
        ax[i, j].patch.set_facecolor('#333')

        plots += 1

plt.show()

# Modelling

In [ ]:
model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=6, nthread=6)

In [ ]:
model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=5)

In [ ]:
preds = model.predict(X_valid)
print(f"XGBoosting Regressor MAE: {mean_absolute_error(preds, y_valid)}")

# What does this mean?
Well. The XGBoost regressor's loss is low. Could this mean that it shows that humans are not really capable of being random? You choose for yourself...